## Data Wrangling Summary

### Objective

Objective is to gather, assess, clean and subsequently briefly analyse the tweet data obtained from [WeRateDogs] twitter channel. The analysis is not meant to be definitive and scientific, especially when it comes to dog ratings actaully given, or dog stages defined within tweets, because the nature of the chanel is not to be ultimate judges.  

[WeRateDogs]: https://twitter.com/dog_rates

Of course, if someone would desparately want their dog's picture to be widely shared, they could certainly use this data to try and infer what is popular with the community and what is not, though it seems kind of evil to game the community in this way.


### Data sources
The following three data sources were used:

1. The WeRateDogs Twitter archive, provided as a file on hand, which was to be downloaded manually.

2. The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) was hosted on Udacity's and was downloaded programmatically here: <https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv>

3. Each tweet's retweet count and favorite ("like") count at minimum, and any additional data was to be downloaded using the tweet IDs in the WeRateDogs Twitter archive, by querying the Twitter API for each tweet's JSON data using Python's Tweepy library, and stored line by line (each line in JSON format) to new file called 'tweet_json.txt' file. 

### Gathering data

> Full operations in `wrangle_act.ipynb`

The WeRateDogs Twitter archive was provided as file, so this is to be read streight from the file. The archive contained 2356 tweets of which 181 retweets (note, deleting retweets was one of last steps of data cleaning, because no reason not to gather and save all data first, this particular load of tweets was not going to impact the speed/memory in a significant way).

The image classification file was provided as URL, and read straight into pandas dataframe with pd.read_csv, using tab separator '\t'. Quick immediate examination showed 2075 entries and no null entries, thus short of 2356 tweets in the archive and 2335 tweets that were actually dowloaded.

Set up Twitter Developer account, set up the app, received the tokens and keys, then connected to the API (set **wait_on_rate_limit** to **True**, **wait_on_rate_limit_notify** to **True**) and queried all the tweets. Added a break in case erorrs exceeded 100 (so that all 2000+ tweets are not queried if there are large number of errors). 

There were 21 errors, all caused by 'No status found with that ID.', which is likely a result of those tweets being subsequently delted, and remaining 2335 tweets were all successfully downloaded and recorded (into 'tweet_json.txt' file).

From 'tweet_json.txt' created a dataframe with the following columns:
> ['tweet_id', 'text_check', 'dog_picture_url', 'tweet_web_url', 
                                      'favourite_count', 'retweet_count', 'retweet_status_id_api']
                                      
Note that 'dog_picture_url', 'tweet_web_url', 'retweet_count' where, if not null, drawn from main dictionary of record of relevant JSON, and if null, drawn from the retweet sub-dictionary of record within JSON.  At this point it was not clear to me that if this information did not exist in the main body, it is a retweet, and it will be later deleted, however, in any case, it produced a more complete dataframe. **Anything that was not found in JSON was labelled as 'None'. **

- Few quick checks were done: for duplicates (none), for missing dog picture urls (197), count of not retweets (2170), missing tweet urls (none), favourite counts/retweet counts/tweet text (no Nones, all sensible).

- Code snipped created to call a JSON for individual tweet according to its ID.

- Code snippet created to call first tweet that has a non null retweet_status (needed when examining how retweet JSONs differ from non-retweet JSONs).

All API tweet data (in JSON, line by line) recorded in `'tweet_json.txt'` and backed up to `'tweet_json_mastercopy.txt'`.

Dataframe with extracted tweet data saved to `'twitter_status_dogs.csv'`.

### Assessing/Cleaning data

> Full operations in `wrangle_act.ipynb`

The data was called into the following dataframes:
    - `df_archive`  - from 'twitter-archive-enhanced.csv'
    - `df_class`    - from 'twitter_image_class.csv'
    - `df_status`   - from 'twitter_status_dogs.csv'

Substantial part of assessing and cleaning was due to the fact that the tweet archive provided (in `df_archive`) could not be fully relied on as accurate, and therefore comparisons had to be carried out between the archive data and the data queried from the twitter API (in `df_status`).

The detailed tidying and cleaning steps full list is in `wrangle_act.ipynb`:

- First, copies of the data propr to cleaning in `archive_clean`,`status_clean`,`class_clean`

- Ordered by execution, in total 2 tidyness issues addressed and 11 cleanliness issued.

- Note that also each "cleanliness" issue that dealt with comparing two columns containing the same information, eg issues #3, #4, #5, #7, #9 had tidyness resolved as well, because they were dealing with ensuring that **each variable formed a column**, and only that one column (avoiding the same variable being duplicated in second column).

On final two tables created (`tweet_clean` and `class_clean`):

- joining `archive_clean` and `status_clean`, to create `tweet_clean`: after it was cleaned, it complied with tidyness protocol, hence no reason to keep it in two separate tables; 
> sidenote, after cleaning it did transpire that 'dog stages' may not be unique for each tweet (but each tweet may contain several dogs with several stages, or one dog could be in two stages), so it could make sense to have a separate table matching tweets to stages as this is many-to-many relationship; but it was already beyond the scope of this project. 

- `class_clean`  It seems that the image classification table `class_clean` (previously `df_class`) could also be joined (to `tweet_clean`) without causing tidyness issues, that said, since it is such a different "unit" of information, with its own independent source (ie classification model output), it was kept separate.

### Data analysis

> Full operations in `wrangle_act.ipynb`

> Full analysis report in `act_report.ipynp`

Analysis consisted of quick examination of:
- correlations between numerical variables:
> a positive correlation between retweet_count and favourite_count was found (as would be expected)
- visual inspection of how the correlation between retweet_count and favourite_count differs/evolves for each subset of dogs/tweets that are reted respectively from 9/10 to 13/10 (there are five most pupulous categories)
> a positive correlation (between favourite and retweet counts) was observed for each rating; and it's clear that the extreme number of favourite and retweet counts have occured mostly for higher ratings (i.e. 12/10 and 13/10 in the numerator)
- for 10 most popular breeds, what is the distribution of their favourite counts
- for each dog stage, what is the distribution of their favourite counts
- last but not least, calling the Top Dogs (ie top 100 favourte ('like') counts) picture, tweet text, and like/retweet stats